# Method 1

In [66]:
# Create data frames from the source tables 
prd_feature = spark.read.parquet("s3://imba-kevin0019/features/prd_feature_db/")
prd_feature.show(10)
prd_feature.printSchema()
print("Count: ", prd_feature.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+-----------------+-----------+------------------+
|product_id|prod_second_orders|prod_first_orders|prod_orders|reordered_reorders|
+----------+------------------+-----------------+-----------+------------------+
|     39928|              7798|            14376|      50141|             35765|
|      4421|              2366|             5529|      12230|              6701|
|     28769|               590|             1687|       3224|              1537|
|     12879|               237|              495|       1293|               798|
|     33700|                59|              360|        456|                96|
|     48015|               432|              837|       2665|              1828|
|     47144|              5343|             9893|      34583|             24690|
|     21299|                19|               75|        121|                46|
|     14148|                37|              346|        402|                56|
|     35851|               5

In [67]:
up_features = spark.read.parquet("s3://imba-kevin0019/features/up_feature_db")
up_features.show(10)
up_features.printSchema()
print("Count: ", up_features.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----------------+----------------+--------------+----------------------+
|user_id|product_id|order_number_max|order_number_min|number_ordered|add_to_cart_order_mean|
+-------+----------+----------------+----------------+--------------+----------------------+
|  13183|     21903|              63|               2|            26|    6.3076923076923075|
|  13183|      6740|              32|               9|             3|     4.333333333333333|
|  13185|     41716|              15|              15|             1|                   9.0|
|  13189|     13575|              30|               3|            14|    2.7857142857142856|
|  13191|     33731|              28|              17|             2|                  15.0|
|  13204|     44910|              15|               1|            14|     6.785714285714286|
|  13204|      8424|              30|               1|            17|     5.823529411764706|
|  13218|     15251|              27|               4|            19| 

In [68]:
user_features_1 = spark.read.parquet("s3://imba-kevin0019/features/user_feature_1_db")
user_features_1.show(10)
user_features_1.printSchema()
print("Count: ", user_features_1.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+---------------------------+--------------------------+
|user_id|max_order_number|days_since_prior_order_mean|days_since_prior_order_sum|
+-------+----------------+---------------------------+--------------------------+
|     12|               6|                       26.0|                       130|
|     46|              20|          17.36842105263158|                       330|
|     67|              25|          7.458333333333333|                       179|
|    161|              21|                      16.65|                       333|
|    186|               8|         12.714285714285714|                        89|
|    275|              15|         11.071428571428571|                       155|
|    282|               4|                       18.0|                        54|
|    323|              72|         2.4507042253521125|                       174|
|    336|              12|          4.363636363636363|                        48|
|    350|       

In [69]:
user_features_2 = spark.read.parquet("s3://imba-kevin0019/features/user_feature_2_db")
user_features_2.show(10)
user_features_2.printSchema()
print("Count: ", user_features_2.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------------+-------------------------+------------------+
|user_id|product_id_count|product_id_count_distinct|user_reorder_ratio|
+-------+----------------+-------------------------+------------------+
|  67331|             102|                       68|0.5396825396825397|
|  67353|              39|                       16|0.7931034482758621|
|  67465|             450|                      119| 1.402542372881356|
|  68973|             394|                      227|0.6028880866425993|
|  69180|             201|                       97|0.7222222222222222|
|  69403|              34|                       20|              0.56|
|  64856|             419|                      170|1.3681318681318682|
|  65438|             432|                      120|1.3109243697478992|
|  65853|             385|                      132|1.1933962264150944|
|  66171|             551|                      248|1.8819875776397517|
+-------+----------------+-------------------------+------------

In [70]:
# Join the data frames 
joinDF = ((up_features.join(prd_feature, "product_id")).join(user_features_1, "user_id")).join(user_features_2, "user_id")
joinDF.show(10)
joinDF.printSchema()
print("Count: ", joinDF.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+----------------+----------------+--------------+----------------------+------------------+-----------------+-----------+------------------+----------------+---------------------------+--------------------------+----------------+-------------------------+------------------+
|user_id|product_id|order_number_max|order_number_min|number_ordered|add_to_cart_order_mean|prod_second_orders|prod_first_orders|prod_orders|reordered_reorders|max_order_number|days_since_prior_order_mean|days_since_prior_order_sum|product_id_count|product_id_count_distinct|user_reorder_ratio|
+-------+----------+----------------+----------------+--------------+----------------------+------------------+-----------------+-----------+------------------+----------------+---------------------------+--------------------------+----------------+-------------------------+------------------+
|  13183|     21903|              63|               2|            26|    6.3076923076923075|             36557|    

In [71]:
# Write out a single file to S3 directory "finaltable"
singleDF = joinDF.repartition(1)
singleDF.write.csv("s3://imba-kevin0019/features/features_join", header = "true")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Method 2

In [72]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [73]:
prd_feature = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"path": ["s3://imba-kevin0019/features/prd_feature_db/"]})
prd_feature.printSchema()
print("Count: ", prd_feature.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- product_id: long
|-- prod_second_orders: int
|-- prod_first_orders: int
|-- prod_orders: int
|-- reordered_reorders: int

Count:  49677

In [77]:
up_features = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"path": ["s3://imba-kevin0019/features/up_feature_db"]})
up_features.printSchema()
print("Count: ", up_features.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- user_id: long
|-- product_id: long
|-- order_number_max: long
|-- order_number_min: long
|-- number_ordered: long
|-- add_to_cart_order_mean: double

Count:  13307953

In [79]:
user_features_1 = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"path": ["s3://imba-kevin0019/features/user_feature_1_db/"]})
user_features_1.printSchema()
print("Count: ", user_features_1.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- user_id: int
|-- max_order_number: int
|-- days_since_prior_order_mean: double
|-- days_since_prior_order_sum: int

Count:  206209

In [80]:
user_features_2 = glueContext.create_dynamic_frame_from_options(connection_type = "parquet", connection_options = {"path": ["s3://imba-kevin0019/features/user_feature_2_db/"]})
user_features_2.printSchema()
print("Count: ", user_features_2.count())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
|-- user_id: long
|-- product_id_count: int
|-- product_id_count_distinct: int
|-- user_reorder_ratio: double

Count:  206209

In [81]:
# join user features together
users = Join.apply(user_features_1.rename_field('user_id','user_id1'), user_features_2, 'user_id1', 'user_id').drop_fields(['user_id1'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# join everything together
df = Join.apply(Join.apply(up_features, 
                  users.rename_field('user_id','user_id1'), 
                  'user_id','user_id1').drop_fields(['user_id1']),
      prd_feature.rename_field('product_id','product_id1'), 
      'product_id','product_id1').drop_fields(['product_id1'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# convert glue dynamic dataframe to spark dataframe
df_spark = df.toDF()
df_spark.repartition(1).write.mode('overwrite').format('csv').save("s3://imba-kevin0019/features/features_join", header = 'true')

In [87]:
df_spark = df.toDF()
df_spark.show(10)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------------+----------+----------------+--------------------------+----------------+------------------+----------------+----------------------+--------------+------------------+-------------------+-----------+---------------------------+-----------------+-------+----------------+
|product_id_count_distinct|product_id|product_id_count|days_since_prior_order_sum|max_order_number|reordered_reorders|order_number_min|add_to_cart_order_mean|number_ordered|prod_second_orders| user_reorder_ratio|prod_orders|days_since_prior_order_mean|prod_first_orders|user_id|order_number_max|
+-------------------------+----------+----------------+--------------------------+----------------+------------------+----------------+----------------------+--------------+------------------+-------------------+-----------+---------------------------+-----------------+-------+----------------+
|                      177|     37217|             313|                       262|              15|             